In [2]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd



# Seed para que las redes con iguales parametros no generen resultados aleatorios y tener repetibilidad
np.random.seed(42)
tf.random.set_seed(42)

# Para las graficas
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

#Función para plotear
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

#Función Rounded Accuracy
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

    
#Traemos los datos de Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full_normalized = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full_normalized[:-5000], X_train_full_normalized[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]
#Reshape de las imagenes de fashion MNIST guardadas en X_train para que esten centradas en 0 
X_train = X_train.reshape(-1, 28, 28, 1) * 2. - 1. 

#Función para ver los resultados de las reconstrucciones
def show_reconstructions(model, images=X_test, n_images=5):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])
        
#Función para ver los resultados de las reconstrucciones en el modelo con 2 salidas
def show_reconstructions_class(model, images=X_test, n_images=5):
    reconstructions,_ = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])
        
#Función para visualziar multiples imagenes generadas por las GANs o VAEs        
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

2024-05-28 21:18:11.453511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 21:18:11.453632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 21:18:11.623200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Difinición y Entrenamiento**

In [19]:
#Definimos la red GAN
#Seed para el factor aleatorio
tf.random.set_seed(42)
np.random.seed(42)

#Esta variable marca el shape del input del generador
codings_size = 100

#Definimos el generador como un modelo secuencial
generator = keras.models.Sequential([
    keras.layers.InputLayer(shape=[codings_size]),
    keras.layers.Dense(7 * 7 * 128),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=1, padding="SAME",
                                 activation="relu",kernel_initializer="HeNormal",kernel_regularizer=keras.regularizers.l2(0.015)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2DTranspose(32, kernel_size=5, strides=2, padding="SAME",
                                 activation="relu",kernel_initializer="HeNormal",kernel_regularizer=keras.regularizers.l2(0.015)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="SAME",
                                 activation="tanh"),
])

#Definimos el discriminador como un modelo secuencial
discriminator = keras.models.Sequential([
    keras.layers.InputLayer(shape=[28,28,1]),
    keras.layers.RandomFlip(mode="horizontal"),
    keras.layers.RandomContrast(factor=0.2),
    keras.layers.RandomBrightness(factor=0.2),
    keras.layers.Conv2D(64, kernel_size=5, strides=1, padding="SAME",
                        kernel_initializer="HeNormal",kernel_regularizer=keras.regularizers.l2(0.015)),
    keras.layers.LeakyReLU(0.3),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(128, kernel_size=5, strides=1, padding="SAME",
                        kernel_initializer="HeNormal"),
    keras.layers.LeakyReLU(0.3),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])

#Creamos el modelo GAN juntanto del generador y el discriminador
gan = keras.models.Sequential([generator, discriminator])
gan.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_15 (Sequential)      │ ?                      │       891,265 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_16 (Sequential)      │ ?                      │       213,633 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,104,898 (4.21 MB)

 Trainable params: 1,104,066 (4.21 MB)

 Non-trainable params: 832 (3.25 KB)

In [20]:
#Visualizamos la estructura tanto del generador como del discriminador
generator.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 6272)           │       633,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_6 (Reshape)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_15             │ (None, 7, 7, 64)       │       204,864 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_16             │ (None, 14, 14, 32)     │        51,232 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_17             │ (None, 28, 28, 1)      │           801 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 891,265 (3.40 MB)

 Trainable params: 890,817 (3.40 MB)

 Non-trainable params: 448 (1.75 KB)

In [21]:
discriminator.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_5 (RandomFlip)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_1               │ (None, 28, 28, 1)      │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness_1             │ (None, 28, 28, 1)      │             0 │
│ (RandomBrightness)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 213,633 (834.50 KB)

 Trainable params: 213,249 (833.00 KB)

 Non-trainable params: 384 (1.50 KB)

In [22]:
#Compilamos el discriminador para modificar su parametro trainable y que solo se entrene cuando se llame directamente su fit 
#o train_on_batch y no al entrenar la GAN completa, si se llama el fit de GAN solo se entrena el generador
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001))
discriminator.trainable = False
#Compilamos el GAN
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [23]:
#Definimos el número de epocas, el batch size y el tamaño del ruido gaussiano que va a usar el generador como base
epochs = 100 
batch_size = 128
noise_shape=codings_size

In [ ]:
#Clear session para no conservar datos de entrenamientos pasados
keras.backend.clear_session()
#Definimos el entrenamiento de la GAN considerando la competencia entre el generador y el discriminador
#Estas listas permiten almacenar los valores de loss para poder hacer graficas luego del entrenamiento
loss=[]
g_loss=[]
d_loss=[]
with tf.device('/gpu:0'):
 for epoch in range(epochs):
    print(f"Currently on Epoch {epoch+1}")
    
    
    for i in range(X_train.shape[0]//batch_size):
        
        if (i+1)%50 == 0:
            print(f"\tCurrently on batch number {i+1} of {X_train.shape[0]//batch_size}")
            
        #Generamos el ruido necesario para el generador
        noise=np.random.normal(size=[batch_size,noise_shape])
        #Usamos el generador para crear imagenes falsas desde el ruido
        gen_image = generator.predict_on_batch(noise)
        #Separamos el dataset en batches
        train_dataset = X_train[i*batch_size:(i+1)*batch_size]
       
        #Entrenamos el discriminador en imagenes reales
        train_label=np.ones(shape=(batch_size,1))
        discriminator.trainable = True
        d_loss_real=discriminator.train_on_batch(train_dataset,train_label)
        
        #Entrenamos el discriminador en imagenes falsas
        train_label=np.zeros(shape=(batch_size,1))
        d_loss_fake=discriminator.train_on_batch(gen_image,train_label)
        #Combinamos los loss de ambos entrenamientos del discriminador 
        d_=.5*d_loss_real+0.5*d_loss_fake
        #Entrenamos el generador
        noise=np.random.normal(size=[batch_size,noise_shape])
        train_label=np.ones(shape=(batch_size,1))
        
        
        #Finalmente entrenamos la gan como conjunto, asegurando que el discriminador no se va a entrenar en esta fase
        discriminator.trainable = False 
        d_g_loss_batch =gan.train_on_batch(noise, train_label)
        #Guardamos los valores de loss
        d_loss.append(d_)
        g_loss.append(d_g_loss_batch)
        batch_loss=.5*d_+.5*d_g_loss_batch
        loss.append(batch_loss)
        
    print("Loss: ",loss[-1])
    #Al inicio y cada 10 epocas se plotean imagenes para ir monitoreando el desempeño de la gan
    if epoch % 10 == 0:
        samples = 10
        x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples, 100)))

        for k in range(samples):
            plt.subplot(2, 5, k+1)
            plt.imshow(x_fake[k].reshape(28, 28), cmap='gray')
            plt.xticks([])
            plt.yticks([])

        plt.tight_layout()
        plt.show()

        
        
print('Training is complete')


Currently on Epoch 1
	Currently on batch number 50 of 429
	Currently on batch number 100 of 429


Exception ignored in: <function _xla_gc_callback at 0x7c5d1fbc3490>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


	Currently on batch number 150 of 429


**Evolución del loss**

In [ ]:
plt.plot(discriminator_loss,label="Discriminator Loss")
plt.plot(generator_loss,label="Generator Loss")
plt.grid(True)
#plt.xlim(0,20)
#plt.ylim(0,1.5)
plt.legend()
plt.show()

**Predicción y visualización de resultados**

In [ ]:
#Otra vez la seed para el tf.random
tf.random.set_seed(42)
np.random.seed(42)

#Le entregamos ruido al generador para que nos muestre que imagenes genera a partir de ese ruido
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator(noise)
#Graficamos las imagenes generadas por el generador
plot_multiple_images(generated_images, 8)

**Discusión**
* Este modelo es increiblemente pesado de ejecutar. Consume tantos recursos que lo máximo que me permite la RAM de kaggle son 10 epocas y tengo que reiniciar l sesión porque la RAM queda llena y no se como vaciarla.
* El problema de ejecutar pocas epocas es que no hay mucho margen para ver la mejora del modelo y los tiempos de ejecución siempre son muy elevados.
* A diferencia de otros modelos, la convergecia de la GAN es compleja. Debido a la constante competencia entre generador y discriminador, las metricas tienden a fluctuar mucho paso tras paso. Si el generador mejora en una epoca tendra un loss bajo pero aumentara el loss del discriminador, y viceversa.

# Pruebas

In [6]:
#Definimos una función para realizar el entrenamiento de la GAN considerando la competencia entre el generador y el discriminador
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=5):
    generator, discriminator = gan.layers
    #Para graficar el loss cree estas 2 listas
    discriminator_loss=[]
    generator_loss=[]
    with tf.device('/gpu:0'):
        for epoch in range(n_epochs):
            print("Epoch {}/{}".format(epoch + 1, n_epochs))
            for i_,X_batch in enumerate(dataset):
                print(f'\r{i_+1}/{round(X_train.shape[0]/X_batch.shape[0])}',end='')
                # phase 1 - training the discriminator on real images
                noise = tf.random.normal(shape=[batch_size, codings_size])
                generated_images = generator.predict_on_batch(noise)
                y1 = tf.constant([[1.]] * batch_size)
                discriminator.trainable = True
                #Guardo en una variable porque train_on_batch retorna el valor del loss para ese batch
                d_loss_real=discriminator.train_on_batch(X_batch, y1)
                #phase 2 - trining the discriminator on fake images
                y2 = tf.constant([[0.]] * batch_size)
                #Guardo en una variable porque train_on_batch retorna el valor del loss para ese batch
                d_loss_fake=discriminator.train_on_batch(generated_images, y2)
                #Y esos valores guardado los meto a la lista que se va a usar para graficar el loss
                d_loss=0.5*d_loss_real+0.5*d_loss_fake
                discriminator_loss.append(d_loss)
                # phase 3 - training the generator
                noise = tf.random.normal(shape=[batch_size, codings_size])
                y3 = tf.constant([[1.]] * batch_size)
                discriminator.trainable = False
                #Similar al discriminador, guardo el return de train_on_batch 
                x=gan.train_on_batch(noise, y3)
                #Y lo agrego a la lista de los losses para graficarlos
                generator_loss.append(x)
            #Mostramos los valores actuales de loss para cada parte del modelo
            print(" Discriminator loss: ",discriminator_loss[-1],". Generator_loss: ",generator_loss[-1])
        #Ploteo 6 imagenes generadas pero solo al final para reducir el espacio usado por la celda de entrenamieno
        plot_multiple_images(generated_images, 6)
        plt.show()
    #Retorno las listas con los losses para hacer las gráficas
    return discriminator_loss,generator_loss

#Definimos el optimzador para customizar el learning rate
#Compilamos el discriminador para modificar su parametro trainable y que solo se entrene cuando se llame directamente su fit 
#o train_on_batch y no al entrenar el GAN completo, si se llama el fit de GAN solo se entrena el generador
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam())
discriminator.trainable = False
#Compilamos el GAN
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam())#Aparntemente va bien pero necesitaria las 10 epocas y ver que pasa, igual demosle la chance y se le pude acomodar el lr

#Siguen un par de procesos para organizar las imagenes de muestra antes de entregarselas al modelo

#Reshape de las imagenes de fashion MNIST guardadas en X_train para que esten centradas en 0 
X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1. 
#Definimos el batch_size que se va a usar en el entrenamiento
batch_size = 64
#Se crea un dataset para generar los batches usando las imagenes con reshape
dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
#Se le hace shuffle para evitar que el modelo aprenda de un patron secuencial
dataset = dataset.shuffle(1000)
#Y finalmente se crean los batches con el size que ya se indico y eliminando los elemenos del dataset que queden
#sobrantes
#Tambien se usa prefetch para aumentar la eficiencia del modelo preparando las imagenes para el proximo train step
#mientras se sigue ejecutando el actual
dataset = dataset.batch(batch_size, drop_remainder=True)

In [7]:
#Clear session para no conservar datos de entrenamientos pasados
keras.backend.clear_session()

#Con todo lo anterior ya podemos llamar la función train y entrenar el modelo
#Recordamos que train_gan retorna 2 listas con los losses que podemos usar para graficar su evolución
loss_1,loss_2=train_gan(gan, dataset, batch_size, codings_size,n_epochs=30)

Epoch 1/30
1/859

W0000 00:00:1716234243.115995      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


250/859

KeyboardInterrupt: 

In [ ]:
plt.plot(discriminator_loss,label="Discriminator Loss")
plt.plot(generator_loss,label="Generator Loss")
plt.grid(True)
#plt.xlim(0,20)
#plt.ylim(0,1.5)
plt.legend()
plt.show()

In [ ]:
#Otra vez la seed para el tf.random
tf.random.set_seed(42)
np.random.seed(42)

#Le entregamos ruido al generador para que nos muestre que imagenes genera a partir de ese ruido
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator(noise)
#Graficamos las imagenes generadas por el generador
plot_multiple_images(generated_images, 8)